In [ ]:
# install the follwoing packages in the enviroment:
# python3 -m pip install pandas
# python3 -m pip install seaborn

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import numpy as np
import json

import os

## Hypotheses to be tested

#### As reference values to compare to, we will use demographics from ECS from agu, egu, ssa. This will give an idea of how many of the active scientists at these professional levels are represented to the peer-reviewed articles (which is the main crucial factor for career advancing and perhaps the daily goal of most academics)

FIRST GLANCES AT DATA

- % of female first authors (hists?)
- % publications with all male vs. % publications with all female authors (hists?)

CO-AUTHORSHIP ANALYSES

- When 1st author is female: % of male vs. female co-authors (bars..?)
- When 1st author is female: likelihood of last author (possibly PI) to be female vs. male 
- When 1st author is male: % of female coauthors and % of male coauthors
- When the last name is female (possible PI), is there a higher % of female co-authors vs. male ones?

JOURNAL IF ANALYSES:

- Correlation between IF and female first authors: does higher IF mean fewer female first authors?




In [ ]:
# Define local paths

root = ! pwd
root = root[0]

RAW_DIR=root+"/author_allgenders/"  # TODO: change this with the final json files obtained from parsing

if not os.path.exists(RAW_DIR):
    print("The directory {} does not exist.\nThere is no raw data for statistical analysis.".format(RAW_DIR))

In [ ]:
## read all json files and create a dataframe

dfs = [] 

for _, _, files in os.walk(RAW_DIR):
    for file in files:
        with open(RAW_DIR+file) as f:
            data = json.loads(f.read())
            df = pd.json_normalize(data)
            df = df.drop(columns=['first_author','all_names', 'year', 'month', 'title', 'id'])
            dfs.append(df)

df = pd.concat(dfs, ignore_index=True)
df

In [ ]:
# clean some journal names

df.loc[df.journal=='E%26PSL','journal'] = 'EPSL'

df.loc[df.journal.str.contains("Bulletin"),'journal'] = 'BSSA'

df.loc[df.journal.str.contains("Seismological"),'journal'] = 'SRL'

### Create new columns in the dataframe extracting useful information from list of coauthors

In [ ]:
# Number of authors:

df['Number_authors'] = df['all_genders'].apply(lambda x: len(x)) #take the length of the list all_genders

# First author's gender and percentage:

df['First_Author_gend'] = df['all_genders'].apply(lambda x: x[0]) #take the first element of the list all_genders
df['First_Author_perc'] = df['all_percent'].apply(lambda x: x[0])

# Last author's gender and percentage:

df['Last_Author_gend'] = df['all_genders'].apply(lambda x: x[-1]) #take the last element of the list all_genders
df['Last_Author_perc'] = df['all_percent'].apply(lambda x: x[-1])

df

 #### It is easier if the all probabilities are with respect to the same gender (female)

In [ ]:
# prob(female) = 1 - prob(male)

# Prob last author female:

df['Last_Author_probF'] = df['Last_Author_perc']
df.loc[df['Last_Author_gend'] == 'male','Last_Author_probF'] = \
    1 - df.loc[df['Last_Author_gend'] == 'male','Last_Author_probF']

# Prob first author female:

df['First_Author_probF'] = df['First_Author_perc']
df.loc[df['First_Author_gend'] == 'male','First_Author_probF'] = \
    1 - df.loc[df['First_Author_gend'] == 'male','First_Author_probF']

df

## Now we can compute some interesting probabilities:

### Useful formulas:

Suppose $x_i$ refers to the article $i$ and $N$ is the total number of articles. Then, the probability of an article having female author is:

$$p(\text{female}) = \sum_{i}^N p(\text{female}|x_i) p(x_i). $$

If we have all the probabilities with respect to the female gender, then the probability of having a male author will be:

$$p(\text{male}) = \sum_{i}^N (1 - p(\text{female}|x_i)) p(x_i). $$

$p(x_i)$ is the probability of the article $x_i$. All articles have the same probability, therefore $p(x_i) = \frac{1}{N}$. This means that the formulas above are same as taking the average of  $p(\text{female}|x_i)$ or $(1 - p(\text{female}|x_i))$, respectively.


### Let's compute some easy statistics to start

In [ ]:
print('Probability of having a female first author:', df['First_Author_probF'].sum()/df.shape[0])
print('Probability of having a male first author:', (1 - df['First_Author_probF']).sum()/df.shape[0])


print('Probability of having a female last author:', df['Last_Author_probF'].sum()/df.shape[0])
print('Probability of having a male last author:', (1 - df['Last_Author_probF']).sum()/df.shape[0])



#### Probabilities of having at least one male/female author in an article

Having at least one female author refers to any coauthor combination excluding the case in which all authors are male:

$$p(\text{at least 1 female}|x_i) = 1 - p(\text{all male}|x_i)$$

Computing probability for all male coauthors is easier. In the following, we drop the dependency on $x_i$ for clarity.

$$p(\text{all male}) = p(\text{male}_1)p(\text{male}_2|\text{male}_1)p(\text{male}_3|\text{male}_1,\text{male}_2)... = \prod_i^n p(\text{male}_i)$$

where n is the number of authors and the last step assumes that the gender probability of each authorship is independent of the gender of other coauthors (just to simplify the problem). 

In [ ]:
#Define functions to multiply probabilities in each row

#prob at least a female author

def Prob_atleast_Fauthor(x,y):
    prod = 1
    for i,elem in enumerate(x):
        if elem == 'male':
            prod *= float(y[i]) 
        elif elem == 'female':
            prod *= 1 - float(y[i])
    return 1 - prod

#prob at least a male author

def Prob_atleast_Mauthor(x,y):
    prod = 1
    for i,elem in enumerate(x):
        if elem == 'male':
            prod *= 1 - float(y[i]) 
        elif elem == 'female':
            prod *= float(y[i])
    return 1 - prod


# Create corresponding columns:

df['Prob_atleast_Fauthor'] = df.apply(lambda x: Prob_atleast_Fauthor(x.all_genders, x.all_percent), axis=1)
df['Prob_atleast_Mauthor'] = df.apply(lambda x: Prob_atleast_Mauthor(x.all_genders, x.all_percent), axis=1)

df

In [ ]:
print('Probability of having at least one female author in an article', 
      df['Prob_atleast_Fauthor'].sum()/df.shape[0])

print('Probability of having at least one male author in an article', 
      df['Prob_atleast_Mauthor'].sum()/df.shape[0])

print('or the opposite...')

print('Probability of having at all female authors in an article', 
      1 - df['Prob_atleast_Mauthor'].sum()/df.shape[0])

print('Probability of having at all male authors in an article', 
      1 - df['Prob_atleast_Fauthor'].sum()/df.shape[0])

#### We can compute the same quantity per each journal

In [ ]:

journals = df['journal'].unique() # a list of unique journal names

df['P_atleast_F_journal'] = df['Prob_atleast_Fauthor'] #initialize the columns
df['P_atleast_M_journal'] = df['Prob_atleast_Mauthor']

for i in journals: #update values for each journal
    cond = df['journal']==i
    df.loc[cond,'P_atleast_F_journal'] = df.loc[cond,'Prob_atleast_Fauthor'].sum()/df[cond].shape[0]
    df.loc[cond,'P_atleast_M_journal'] = df.loc[cond,'Prob_atleast_Mauthor'].sum()/df[cond].shape[0]
    

##### Make bar plots

In [ ]:
sns.barplot(y="journal", x="P_atleast_F_journal",  data=df)

In [ ]:
sns.barplot(y="journal", x="P_atleast_M_journal",  data=df)

#### We should check if the numbers above are biased or are a consequence of female/male author distribution.

We can generate synthetic data using the distribution of female/male authors.


#### 1. Probability of an author being female/male:

In [ ]:
def Prob_Fauthor(x,y):
    sum = 0
    for i,elem in enumerate(x):
        if elem == 'male':
            sum += 1 - float(y[i]) 
        elif elem == 'female':
            sum += float(y[i])
    return sum

def Prob_Mauthor(x,y):
    sum = 0
    for i,elem in enumerate(x):
        if elem == 'male':
            sum += float(y[i]) 
        elif elem == 'female':
            sum += 1 - float(y[i])
    return sum

df['Prob_Fauthor'] = df.apply(lambda x: Prob_Fauthor(x.all_genders, x.all_percent), axis=1)
df['Prob_Mauthor'] = df.apply(lambda x: Prob_Mauthor(x.all_genders, x.all_percent), axis=1)

df

#### 2. Create some synthetic genders for authors:

In [ ]:
### Define a random sampler from the distribution of female/male authors:

elements = ['female', 'male', 'init']
p1 = df['Prob_Fauthor'].sum()/df['Number_authors'].sum() #prob of an author being female
p2 = df['Prob_Mauthor'].sum()/df['Number_authors'].sum() #prob of an author being male
p3 = 1 - p1 - p2 #prob of an author being init

probabilities = [p1, p2 , p3]
np.random.choice(elements, 10, p=probabilities) # example: take 10 random samples

In [ ]:
# Generate synthetic genders per each article maintaining the number of authors:

df['Synth_genders'] = df['Number_authors'].apply(lambda x: np.random.choice(elements, x, p=probabilities))
df

In [ ]:
### Now check for these synthetics the probabilities:

# prob having at least one female author

def Prob_atleast_Fauthor_synth(x):
    prod = 1
    for i,elem in enumerate(x):
        if elem == 'male':
            prod *= 1 
        elif elem == 'female':
            prod *= 0
    return 1 - prod

df['Prob_atleast_Fauthor_synth'] = df.apply(lambda x: Prob_atleast_Fauthor_synth(x.Synth_genders), axis=1)

print('Probability of having at least one female author in an article', 
      df['Prob_atleast_Fauthor_synth'].sum()/df.shape[0])

# prob having at least one male author

def Prob_atleast_Mauthor_synth(x):
    prod = 1
    for i,elem in enumerate(x):
        if elem == 'male':
            prod *= 0
        elif elem == 'female':
            prod *= 1
    return 1 - prod

df['Prob_atleast_Mauthor_synth'] = df.apply(lambda x: Prob_atleast_Mauthor_synth(x.Synth_genders), axis=1)

print('Probability of having at least one male author in an article',
      df['Prob_atleast_Mauthor_synth'].sum()/df.shape[0])


In [ ]:
### What is the prob of first and last authorships?


df['First_Author_Fperc_synth'] = df['Synth_genders'].apply(lambda x: 1 if x[0]=='female' else 0)
df['First_Author_Mperc_synth'] = df['Synth_genders'].apply(lambda x: 1 if x[0]=='male' else 0)

df['Last_Author_Fperc_synth'] = df['Synth_genders'].apply(lambda x: 1 if x[-1]=='female' else 0)
df['Last_Author_Mperc_synth'] = df['Synth_genders'].apply(lambda x: 1 if x[-1]=='male' else 0)

print('First author female:', df['First_Author_Fperc_synth'].sum()/df.shape[0])
print('First author male:', df['First_Author_Mperc_synth'].sum()/df.shape[0])
print('Last author female:', df['Last_Author_Fperc_synth'].sum()/df.shape[0])
print('Last author male:', df['Last_Author_Mperc_synth'].sum()/df.shape[0])
